In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Label, Layout
import ipywidgets as widgets
import math

In [2]:
# Preset values go here
depth_factor = 0.75 # the assumed 
default_output = ("Yield:  0.000  kilotons\n" +
    "Cavity Radius:  0  meters\n" +
    "Cavity Volume:  0  cubic meters\n" +
    "Minimum burial depth:  0  meters (VERY rough estimate)\n" +
    "Minimum spacing for equivalent test:  30  meters (VERY rough estimate)\n")

test_list = (("North Korea 2006", {'mb': 4.1}),
        ("North Korea 2013", {'mb': 4.9, 'depth': 430}),
        ("North Korea 2016", {'mb': 5.1, 'depth': 673}),
        ("North Korea 2017", {'mb': 6.1, 'depth': 789}),
        ("Novaya Zemlya 1966", {'mb': 6.39}),
        ("Novaya Zemlya 1975", {'mb':6.43}))

test_sites = (("Punggye-ri", {'propagation': 4.25}),
             ("Punggye-ri (with depth correction)", {'propagation': 4.25, 'experimental_depth': True, 'experimental_depth_value': 430}),
             ("Lop Nor", {'propagation': 4.45}),
             ("Nevada", {'propagation': 4.05}),
             ("Novaya Zemlya", {'propagation': 4.3}),
             ("Pokhran", {'propagation': 4.04, 'depth_factor': 0.77}),
             ("Moruroa", {'propagation': 3.71, 'depth_factor': 1.0}),
             ("Chagai", {'propagation': 4.05}),
             ("dry alluvium", {'propagation': 3.75}))

In [3]:
def cavity_radius(kt):
    if kt < 1:
        return 0
    else:
        return 17 * math.pow(math.log10(kt), 1/3)

def cavity_volume(kt):
    return (4/3) * math.pi * math.pow(kt, 3)

def depth_estimate(kt):
    if kt < 1:
        return 0
    else:
        return 120*math.pow(math.log10(kt), 1/3)
    
def spacing_estimate(kt):
    return 2*kt + 30

def yield_estimator(mb, depth=0, depth_factor=0.75, propagation=4.25):
    if(mb==0):
        print(default_output)
        return
    if(mb < 4):
        if(depth==0):
            yld = math.pow(10, (mb-propagation))
        else:
            yld = math.pow(10, (
                mb-(propagation+1.63736)+(0.7875*math.log10(depth))/1.0125))
    else:
        if(depth==0):
            yld = math.pow(10, (mb-propagation)/depth_factor)
        else:
            yld = math.pow(10, (
                mb-(propagation+1.63736)+(0.7875*math.log10(depth))/1.0125))
    
    print("Yield: ", "{0:.3f}".format(yld), " kilotons")
    print("Cavity Radius: ", "{0:.0f}".format(cavity_radius(yld)), " meters")
    print("Cavity Volume: ", "{0:.0f}".format(cavity_volume(yld)), " cubic meters")
    print("Minimum burial depth: ", "{0:.0f}".format(depth_estimate(yld)), " meters (VERY rough estimate)")
    print("Minimum spacing for equivalent test: ", "{0:.0f}".format(spacing_estimate(yld)), 
          " meters (VERY rough estimate)")

In [4]:
mb_input = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=50,
    step=0.1,
    description='Mb: ',
    disabled=False
)
depth_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100000,
    step=1,
    description='Depth (m): ',
    disabled=True
)
prop_text = widgets.BoundedFloatText(
    value=4.25,
    min=3,
    max=5,
    step=0.01,
    description='Propagation: ',
    disabled=False
)
prop_slider = widgets.FloatSlider(
    value=4.25,
    min=3,
    max=5,
    step=0.01,
    description='',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)
use_depth = widgets.Checkbox(
    value=False,
    description='Enable experimental depth calculation?*',
    disabled=False,
    indent=False
)
sites = [(('Custom'), 1)]
for x in range(len(test_sites)):
    sites.append((test_sites[x][0], x+1))
site_selection = widgets.Dropdown(
    options=sites,
    value=1,
    description='Location:'
)
tests = [(('Custom'), 1)]
for x in range(len(test_list)):
    tests.append((test_list[x][0], x+1))
test_selection = widgets.Dropdown(
    options=tests,
    value=1,
    description='Tests:'
)

In [5]:
def toggle_depth(*args):
    depth_input.disabled = not use_depth.value
    if (not depth_input.disabled) and depth_input.value == 0:
        depth_input.value = 250
    else:
        depth_input.value = 0
        
def change_location(*args):
    if site_selection.value == 0:
        prop_text.value = 4.25
        depth_factor = 0.75
    else:
        for config in test_sites[site_selection.value-1]:
            if 'propagation' in config:
                prop_text.value = config.get('propagation')
            if 'depth_factor' in config:
                depth_factor = config.get('depth_factor')
            else:
                depth_factor = 0.75
            if 'experimental_depth' in config:
                use_depth.value = config.get('experimental_depth')
                depth_input.disabled = not config.get('experimental_depth')
                if 'experimental_depth_value' in config:
                    depth_input.value = config.get('experimental_depth_value')
                else:
                    depth_input.value = 250
            else:
                use_depth.value = False
                depth_input.disabled = True
                
def set_test(*args):
    if site_selection.value == 0:
        mb_input.value = 0
    else:
        for config in test_list[test_selection.value-1]:
            if 'mb' in config:
                mb_input.value = config.get('mb')
            if 'depth' in config and use_depth.value is True:
                use_depth.value = True
                depth_input.value = config.get('depth')

# set layout
ui_layout = Layout(
    grid_row='7',
    grid_column='1'
)
grid_order = [test_selection, site_selection, use_depth, mb_input, depth_input, prop_text, prop_slider]

# link any elements
prop_link = widgets.jslink((prop_text, 'value'), (prop_slider, 'value'))

# Nuclear Test Yield Calculator

Input the magnitude (Mb), as well as depth and propagation if available.

Alternatively, choose a preset to see historic nuclear estimates.

In [6]:
# build the final UI
ui = widgets.GridBox(grid_order, layout=ui_layout)

# this is the important run stuff bit
yield_estimator_interactive = widgets.interactive_output(
    yield_estimator, {'mb': mb_input, 'depth': depth_input, 'propagation': prop_text})
use_depth.observe(toggle_depth, 'value')
site_selection.observe(change_location, 'value')
test_selection.observe(set_test, 'value')

# all done
display(ui, yield_estimator_interactive)

GridBox(children=(Dropdown(description='Tests:', options=(('Custom', 1), ('North Korea 2006', 1), ('North Kore…

Output()

\*Only guaranteed for Punggye-ri, see source 2

In [ ]:
# how does this work? todo: apparently you can do LaTeX in Voila?

# Sources & Thanks

This site was built and maintained by [@madwonk](https://twitter.com/madwonk) to streamline work done by [Jeffrey Lewis](https://twitter.com/ArmsControlWonk) and bring it to a larger audience.

## How does this work?
This site is a Jupyter notebook exported with [Voilà](https://github.com/voila-dashboards/voila). The source code is available on [Github](https://github.com/BenMueller/nuclear-testing-calculator)!

## I've found a bug!
Feel free to file a report on Github, or send me a message @madwonk on Twitter.

## Sources
\[1\] [Punggye-ri yield, no depth correction](http://dx.doi.org/10.1785/0120100202)

\[2\] [Punggye-ri with depth correction](https://doi.org/10.1002/grl.50607)

\[3\] [Novaya Zemlya](www.jstor.org/stable/26977)

\[4\] [Pokhran (high estimate from BARC)](http://web.archive.org/web/20010720043355/http://www.barc.ernet.in/webpages/milestones/drs_03.html)

\[5\] [Moruroa](https://apps.dtic.mil/dtic/tr/fulltext/u2/a228258.pdf)

\[6\] [Chagai (corrected for tectonic activity)](https://apps.dtic.mil/dtic/tr/fulltext/u2/a228258.pdf)
